### Abstract

The objective is to choose the best model for a given data, thus we need to make several models  with a portion of the data and evaluating the best model possible by choosing the one with the lowest SSE. After choosing the best model is necessary to create a new model with all training data avaiable in order to create outputs for the real data.

# Import Libraries

In [1]:
import numpy as np
import random
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures,  StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV

# Import Train Data

In [2]:
X_import = np.load("data/Xtrain_Regression1.npy")
Y_import = np.load("data/Ytrain_Regression1.npy")
# create global functions to store the splitted data
X_train,Y_train,X_test,Y_test = 0,0,0,0

# Essencial Functions

In [16]:
from statistics import mean


def split_data(x_import,y_import,test_siz):
    '''Splits the data according to the test_siz fraction'''
    x_train, x_test, y_train, y_test = train_test_split(x_import, y_import, test_size=test_siz)
    return (x_train,y_train,x_test,y_test)


def refresh_data(div):
    '''Refreshes the data split'''
    global X_train,Y_train,X_test,Y_test
    (X_train,Y_train,X_test,Y_test) = split_data(X_import,Y_import,div)

def calc_SSE(y_pred,y):
    '''Calculates the SSE from the predicted data'''
    #return np.sum((y_pred - y)**2)
    return mean_squared_error(y_pred,y)*np.shape(y)[0]

def calc_SSE_beta(x,y,beta):
    '''Calculated the SSE from the X and beta matrix'''
    col_ones = np.ones((np.shape(x)[0],1))
    X = np.hstack([col_ones,x])
    SSE = np.linalg.norm(y - np.matmul(X,beta))**2
    return SSE 

def beta_from_model(model):
    '''Returns the complete beta matrix from the model'''
    beta = model.coef_[0]
    beta[0] = model.intercept_[0]
    return beta 
   
def Average(lst):
    '''Returns the average of a list'''
    return round(sum(lst)/len(lst),2)

# Linear Model

In [ ]:
def linear_model(x, y):
    '''Creates and returns a simple linear regression model of order 1'''
    LR = LinearRegression()
    LR.fit(x,y)
    return LR

def test_linear_model(x_train,y_train,x_test,y_test,iters):
    '''Tests the linear model returning a simple SSE value'''

    SSE_lst = []

    for i in range(iters):
        refresh_data(0.2)
        model_sl = linear_model(X_train, Y_train)
        y_pred = model_sl.predict(X_test)
        SSE_lst.append(calc_SSE(y_pred, Y_test))

    SSE_avg = Average(SSE_lst)
    SSE_std = np.std(SSE_lst)

    return(SSE_avg,SSE_std)

SSE_lin = test_linear_model(X_train,Y_train,X_test,Y_test,500)
print("Average SSE :", SSE_lin[0], "+/-", SSE_lin[1])

# Polynomial Model

In [ ]:
def polynomial_model(x, y, deg):
    '''Creates the polynomial model'''

    poly = PolynomialFeatures(degree=deg, include_bias=True)  
    poly_features = poly.fit_transform(x)  
    poly_reg_model = LinearRegression()  
    poly_reg_model.fit(poly_features, y)
    return poly_reg_model, poly

def test_polynomial_model(deg_max,iters,show_plots=True):
    '''Tests the polynomial model up to deg_max'''

    order_list, SSE, SSE_plt = [i for i in range(1,deg_max+1)], [], []

    def get_avg_sse(i):
        '''Returns the average SSE for a given polynomial degree'''
        SSE_lst = []
        for s in range(1,iters):
            refresh_data(0.2)
            (model,poly) = polynomial_model(X_train,Y_train,i)
            x_test_trans = poly.transform(X_test)
            y_pred = model.predict(x_test_trans)
            SSE_lst.append(calc_SSE(y_pred,Y_test))
        return [Average(SSE_lst), np.std(SSE_lst)]
        

    for i in range(1,deg_max+1):
        SSE.append(get_avg_sse(i))
        SSE_plt.append(SSE[-1][0])
    
    
    if show_plots:
        plt.scatter(order_list, SSE_plt)
        plt.xlabel('order')
        plt.ylabel('SSE')
        plt.title('SSE vs Polynomial Order')
        
        return SSE, plt
    
    else:
        return SSE


SSE_pol, plot_pol = test_polynomial_model(4,500,show_plots=True)
for i in range(len(SSE_pol)):
    print("SSE order",i+1,":",SSE_pol[i][0], "+/-", SSE_pol[i][1])
plot_pol.show()

# Ridge Model

In [ ]:
def ridge_model(x,y,alpha):
    '''This function creates a ridge model'''
    model = Ridge(alpha=alpha)
    model.fit(x,y)
    return model

def ridge_modelcv(
    param = {
    'alpha':[.0001, 0.001,0.01, 0.01,1],
    'fit_intercept':[True,False],
    'normalize':[True,False],
    'solver':['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
       }
    
    model=Ridge()

def test_ridge_model(iters,show_plots=True):
    '''This function tests the ridge model'''





    
    alpha_list = [i for i in np.arange(0.1,0.9,0.05)]
    SSE, std = [], []

    def get_avg_sse(alpha):
        '''Returns the average SSE for a given alpha'''
        SSE_lst = []
        for s in range(1,iters):
            refresh_data(0.2)
            model = ridge_model(X_train,Y_train,alpha)
            y_pred = model.predict(X_test)
            SSE_lst.append(calc_SSE(y_pred,Y_test))
        return [Average(SSE_lst), np.std(SSE_lst)]

    for alpha in alpha_list:
        SSE_std = get_avg_sse(alpha)
        SSE.append(SSE_std[0])
        std.append(SSE_std[1])

    min_SSE = min(SSE)
    min_SSE_index = SSE.index(min_SSE)
    min_alpha = alpha_list[min_SSE_index]
    min_std = std[min_SSE_index]

    if show_plots:    
        plt.scatter(alpha_list, SSE)
        plt.xlabel('alpha')
        plt.ylabel('SSE')
        plt.title('SSE vs Alpha')
        return min_SSE, min_std, min_alpha, plt
    else:
        return min_SSE, min_std, min_alpha

    
SSE_rid, std_rid, alpha_rid, plot_rid = test_ridge_model(1500)
print("Average SSE:", SSE_rid ,"+/-",std_rid, "\nAlpha:",alpha_rid)
plot_rid.show()

In [ ]:
def ridgecv_model(x,y,alph,cvs):
    '''This function creates a ridge model'''
    model = RidgeCV(alpha=alph,cv = cvs, normalize=True,store_cv_values = True)
    model.fit(x,y)
    return model

def test_ridge_model(iters,show_plots=True):
    '''This function tests the ridge model'''
    alpha_list = [i for i in np.arange(0.1,0.9,0.05)]
    SSE, std = [], []

    def get_avg_sse(alpha,cv):
        '''Returns the average SSE for a given alpha'''
        SSE_lst = []
        for s in range(1,iters):
            # refresh_data(0.2)
            model = ridgecv_model(X_train,Y_train,alpha,cv)
            y_pred = model.predict(X_test)
            SSE_lst.append(calc_SSE(y_pred,Y_test))
        return [Average(SSE_lst), np.std(SSE_lst)]

    for alpha in alpha_list:
        SSE_std = get_avg_sse(alpha)
        SSE.append(SSE_std[0])
        std.append(SSE_std[1])

    min_SSE = min(SSE)
    min_SSE_index = SSE.index(min_SSE)
    min_alpha = alpha_list[min_SSE_index]
    min_std = std[min_SSE_index]

    if show_plots:    
        plt.scatter(alpha_list, SSE)
        plt.xlabel('alpha')
        plt.ylabel('SSE')
        plt.title('SSE vs Alpha')
        return min_SSE, min_std, min_alpha, plt
    else:
        return min_SSE, min_std, min_alpha

    
SSE_rid, std_rid, alpha_rid, plot_rid = test_ridge_model(1500)
print("Average SSE:", SSE_rid ,"+/-",std_rid, "\nAlpha:",alpha_rid)
plot_rid.show()

# Lasso Model

In [47]:
def lasso_model(x_train,y_train,a):
    '''Creates a lasso model'''
    model_lasso = Lasso(alpha=a)
    model_lasso.fit(x_train,y_train)
    return model_lasso

def lasso_model_cv(x_train,y_train):
    '''Creates a lassocv model'''
    model_lasso = LassoCV(cv=5, random_state=0, max_iter=10000)
    model_lasso.fit(x_train,y_train)
    return model_lasso

def test_lasso_model():
    '''This function tests the lasso model'''
    refresh_data(0.2)
    scaler = StandardScaler().fit(X_train) 
    x_train = scaler.transform(np.copy(X_train)) 
    x_test = scaler.transform(np.copy(X_test))
    model = lasso_model_cv(x_train,Y_train)
    alpha = model.alpha_
    print("The best alpha:",alpha)
    best_lasso = lasso_model(x_train,Y_train,alpha)
    y_pred = best_lasso.predict(x_test)
    print("SSE:",calc_SSE(y_pred,Y_test))

test_lasso_model()

The best alpha: 0.0038697391767761446
SSE: 0.3595237020398174


C:\Users\ricar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1614: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Conclusions

Let's assume Ridge is the best model due to a marginal difference in SSE

In [ ]:
x_eval_import = np.load("data/Xtest_Regression1.npy")
model = ridge_model(X_import,Y_import,0.25)
y_pred = model.predict(x_eval_import)
y_pred

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=25889058-66de-44c8-ad64-ee695f6c1052' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>